# 🧪 ADK Application Testing

This notebook demonstrates how to test an ADK (Agent Development Kit) application.
It covers both local and remote testing, both with Agent Engine and Cloud Run.

> **Note**: This notebook assumes that the agent files are stored in the `app` folder. If your agent files are located in a different directory, please update all relevant file paths and references accordingly.

## Install dependencies

In [ ]:
!pip install google-cloud-aiplatform --upgrade

### Import libraries

In [1]:
import json

import requests
import vertexai

### Initialize Vertex AI Client

In [2]:
# Initialize the Vertex AI client
#LOCATION = "asia-northeast1"
LOCATION = "us-central1"

client = vertexai.Client(
    location=LOCATION,
)

## If you are using Agent Engine
See more documentation at [Agent Engine Overview](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview)

### Remote Testing

In [3]:
# Set to None to auto-detect from ./deployment_metadata.json, or specify manually
# "projects/PROJECT_ID/locations/us-central1/reasoningEngines/ENGINE_ID"
REASONING_ENGINE_ID = "projects/1045259343465/locations/us-central1/reasoningEngines/5491845076462075904"
#REASONING_ENGINE_ID = "projects/1045259343465/locations/asia-northeast1/reasoningEngines/6397134572261212160"

if REASONING_ENGINE_ID is None:
    try:
        with open("../deployment_metadata.json") as f:
            metadata = json.load(f)
            REASONING_ENGINE_ID = metadata.get("remote_agent_engine_id")
    except (FileNotFoundError, json.JSONDecodeError):
        pass

print(f"Using REASONING_ENGINE_ID: {REASONING_ENGINE_ID}")
# Get the existing agent engine
remote_agent_engine = client.agent_engines.get(name=REASONING_ENGINE_ID)

Using REASONING_ENGINE_ID: projects/1045259343465/locations/us-central1/reasoningEngines/5491845076462075904


Generate test session

In [4]:
session = await remote_agent_engine.async_create_session(user_id="test")
session

{'state': {},
 'userId': 'test',
 'lastUpdateTime': 1762405844.959562,
 'appName': '5491845076462075904',
 'events': [],
 'id': '1238945370218692608'}

Send test query #1

In [5]:
async for event in remote_agent_engine.async_stream_query(
    message="hi! what's the weather in San francisco?", user_id="test", session_id = session['id']
):
    print(event)

{'content': {'parts': [{'thought_signature': 'CrsBAePx_15-nH2DHqr3Lku1kHonmpNvJVo922NfJJ9TaNWrAeVJIaRzIjZWAJKowM8vxXcxVy_msgqQVScX2Mrne07TSe1yqVYGk2ScfngiqGd3xwNw930tIfx8YcQ12zK9OQgfGb2WjAMYAI5ik82YIniSrk-KGmIe6kc8_L6CaNstDXt6ipLn2Hd35IhyGrzPQgw1LwfYsq9krZ09_BX-wSMx3O2iuvAHKn9i1diYx-gGHT60RjPhmu5U_A==', 'function_call': {'id': 'adk-d2c51d8d-3dbe-43ea-8a52-2b0eb98ea63c', 'args': {'query': 'San Francisco'}, 'name': 'get_weather'}}], 'role': 'model'}, 'finish_reason': 'STOP', 'usage_metadata': {'candidates_token_count': 6, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 6}], 'prompt_token_count': 156, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 156}], 'thoughts_token_count': 37, 'total_token_count': 199, 'traffic_type': 'ON_DEMAND'}, 'avg_logprobs': -1.0723079840342205, 'invocation_id': 'e-bc55732e-9225-47c5-8b3b-80027845083f', 'author': 'root_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}, 'requested_tool_conf

Send test query #2

In [6]:
async for event in remote_agent_engine.async_stream_query(
    message="hi! what's the weather in chicago?", user_id="test", session_id = session['id']
):
    print(event)

{'content': {'parts': [{'thought_signature': 'Cq8BAePx_16y4iM4K7IKcygtP3lvEvJdNLkL7xhxl_gOcAktS-rqH0C-Vgd4GDXhxghW8fYPwSyJCutW1QxkraAMaZyt_PMFvoL8vD86TbOSBWeRvdHBVbQaXRXwuZ5cQFe4am0_JBDEoJbjE2_T72uOfIlITb68Y1H_KqxOLYuc4_aT7kzIP1RGwto8NTANmjuE_EkS-EnRmKnhgb3ABvdHvfTTsp2UYGvqA2xFE8X6-A==', 'function_call': {'id': 'adk-e630d854-57e8-4aeb-8702-54bf346fa556', 'args': {'query': 'Chicago'}, 'name': 'get_weather'}}], 'role': 'model'}, 'finish_reason': 'STOP', 'usage_metadata': {'candidates_token_count': 5, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 5}], 'prompt_token_count': 199, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 234}], 'thoughts_token_count': 35, 'total_token_count': 239, 'traffic_type': 'ON_DEMAND'}, 'avg_logprobs': -0.07162601947784424, 'invocation_id': 'e-49a3f551-31eb-4665-991f-35e6397844de', 'author': 'root_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}, 'requested_tool_confirmations': {}}, 'lon

In [ ]:
remote_agent_engine.register_feedback(
    feedback={
        "score": 5,
        "text": "Great response!",
        "invocation_id": "test-invocation-123",
        "user_id": "test",
    }
)

Manually create memory (By default, memory is not automatically generated)

In [ ]:
session = await remote_agent_engine.async_get_session(user_id="test", session_id=session['id'])
await remote_agent_engine.async_add_session_to_memory(session=session)

Search memory using similarity search by query

In [10]:
await remote_agent_engine.async_search_memory(user_id="test", query="SFO")

{'memories': [{'content': {'parts': [{'functionCall': None,
      'functionResponse': None,
      'executableCode': None,
      'fileData': None,
      'inlineData': None,
      'text': 'I asked about the weather in San Francisco.',
      'videoMetadata': None,
      'thoughtSignature': None,
      'codeExecutionResult': None,
      'thought': None}],
    'role': 'user'},
   'author': 'user',
   'timestamp': '2025-11-06T05:11:02.496867+00:00'},
  {'author': 'user',
   'timestamp': '2025-11-06T05:11:02.500434+00:00',
   'content': {'parts': [{'videoMetadata': None,
      'codeExecutionResult': None,
      'executableCode': None,
      'fileData': None,
      'thoughtSignature': None,
      'thought': None,
      'functionResponse': None,
      'text': 'I asked about the weather in Chicago.',
      'inlineData': None,
      'functionCall': None}],
    'role': 'user'}}]}

In [13]:
pager = client.agent_engines.memories.list(name=REASONING_ENGINE_ID)
for i, page in enumerate(pager):
  print(f"========== Memory {i} =========")
  print(f"Expire: {page.expire_time}")
  print(f"TTL: {page.ttl}")
  print(f"Revision Expire: {page.revision_expire_time}")
  print(f"Revision TTL: {page.revision_ttl}")
  print(f"Disable memory revisions: {page.disable_memory_revisions}")
  print(f"Create time: {page.create_time}")
  print(f"Description: {page.description}")
  print(f"Display name: {page.display_name}")
  print(f"Fact: {page.fact}")
  print(f"Name: {page.name}")
  print(f"Scope: {page.scope}")
  print(f"Update time: {page.update_time}")
  print("\n")


========== Memory 0 =========
Expire: None
TTL: None
Revision Expire: None
Revision TTL: None
Disable memory revisions: None
Create time: 2025-11-06 05:11:02.500434+00:00
Description: None
Display name: None
Fact: I asked about the weather in Chicago.
Name: projects/1045259343465/locations/us-central1/reasoningEngines/5491845076462075904/memories/5503288243728154624
Scope: {'app_name': '5491845076462075904', 'user_id': 'test'}
Update time: 2025-11-06 05:11:02.500434+00:00


========== Memory 1 =========
Expire: None
TTL: None
Revision Expire: None
Revision TTL: None
Disable memory revisions: None
Create time: 2025-11-06 05:10:56.973181+00:00
Description: None
Display name: None
Fact: I asked about the weather in San Francisco.
Name: projects/1045259343465/locations/us-central1/reasoningEngines/5491845076462075904/memories/7737073658903920640
Scope: {'app_name': '5491845076462075904', 'user_id': 'test'}
Update time: 2025-11-06 05:11:02.496867+00:00




### Local Testing

You can import directly the AgentEngineApp class within your environment. 
To run the agent locally, follow these steps:
1. Make sure all required packages are installed in your environment
2. The recommended approach is to use the same virtual environment created by the 'uv' tool
3. You can set up this environment by running 'make install' from your agent's root directory
4. Then select this kernel (.venv folder in your project) in your Jupyter notebook to ensure all dependencies are available

In [ ]:
# Uncomment the following lines if you're not using the virtual environment created by uv
# import sys
# sys.path.append("../")

In [18]:
import sys
sys.path.insert(0, '../')
from app.agent import root_agent
from app.agent_engine_app import AgentEngineApp

agent_engine = AgentEngineApp(agent=root_agent)

In [19]:
async for event in agent_engine.async_stream_query(message="hi!", user_id="test"):
    print(event)

INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.VERTEX_AI, stream: False
INFO:google_genai.models:AFC is enabled with max remote calls: 10.
INFO:google_adk.google.adk.models.google_llm:Response received from the model.


{'content': {'parts': [{'text': 'Hello! How can I help you today?\n'}], 'role': 'model'}, 'finish_reason': 'STOP', 'usage_metadata': {'candidates_token_count': 10, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 10}], 'prompt_token_count': 147, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 147}], 'total_token_count': 157, 'traffic_type': 'ON_DEMAND'}, 'avg_logprobs': -0.0402709573507309, 'invocation_id': 'e-ce2befdb-cf58-4a40-b668-0f14d68493fe', 'author': 'root_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}, 'requested_tool_confirmations': {}}, 'id': '6240b211-2065-4dfb-8f06-3057897e7744', 'timestamp': 1762408625.148297}


Delete agent

In [ ]:
client = vertexai.Client(
    location=LOCATION,
)
client.agent_engines.delete(
    name=REASONING_ENGINE_ID,
    force=True, # Optional, if the agent has resources (e.g. sessions, memory)
)